In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("test")
    

<Experiment: artifact_location='mlflow-artifacts:/719300370323682557', creation_time=1716590413114, experiment_id='719300370323682557', last_update_time=1716590413114, lifecycle_stage='active', name='test', tags={}>

In [3]:
import pandas as pd

df = pd.read_parquet('homeworks/hw1/yellow_tripdata_2023-01.parquet', engine='pyarrow')


df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df['duration'].dt.total_seconds() / 60

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

In [4]:
from sklearn.feature_extraction import DictVectorizer

# Convert the dataframe into a list of dictionaries
train_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
dv.fit(train_dicts)

# Get the feature matrix
X_train = dv.transform(train_dicts)

# Get the target vector
y_train = df_filtered.duration.values

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [ ]:
# load the test data

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df_val = read_data('homeworks/yellow_tripdata_2023-02.parquet')

val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts) 

y_pred = lr.predict(X_val)

y_val = df_val.duration.values